### Please Import the COCO dataset before running the code.

In [1]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.5 MB/s eta 0:00:0000:0100:01


In [2]:
import torch
from diffusers import StableDiffusionGLIGENPipeline
from diffusers.utils import load_image

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
# Insert the model in the pipeline
pipe = StableDiffusionGLIGENPipeline.from_pretrained(
    "masterful/gligen-1-4-inpainting-text-box", variant="fp16", torch_dtype=torch.float16
)       
pipe = pipe.to("cuda")

model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

safety_checker/config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

(…)kpoints/scheduler_config-checkpoint.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

pytorch_model.fp16.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--masterful--gligen-1-4-inpainting-text-box/snapshots/d6d957f8d27c40889c0d570a616571a5645c8be3/unet: Error no file named diffusion_pytorch_model.fp16.safetensors found in directory /root/.cache/huggingface/hub/models--masterful--gligen-1-4-inpainting-text-box/snapshots/d6d957f8d27c40889c0d570a616571a5645c8be3/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--masterful--gligen-1-4-inpainting-text-box/snapshots/d6d957f8d27c40889c0d570a616571a5645c8be3/vae: Error no file named diffusion_pytorch_model.fp16.safetensors found in directory /root/.cache/huggingface/hub/models--masterful--gligen-1-4-inpainting-text-box/snapshots/d6d957f8d27c40889c0d570a616571a5645c8be3/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/opt/conda/lib/python3.10/site-packages/

In [4]:
import os
os.mkdir("/kaggle/working/output_images/")

In [ ]:
import os
import json
from PIL import Image

# Define paths to images and annotation file
images_path = "/kaggle/input/coco-2017-dataset/coco2017/train2017"
annotations_file_path = "/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json"

# Load the annotations file
with open(annotations_file_path, 'r') as file:
    annotations_data = json.load(file)

# Mapping of image_id to its corresponding annotations
image_to_annotations = {}
for annotation in annotations_data['annotations']:
    image_id = annotation['image_id']
    if image_id not in image_to_annotations:
        image_to_annotations[image_id] = []
    image_to_annotations[image_id].append(annotation)

# Mapping of category_id to category name
category_id_to_name = {
    category['id']: category['name'] for category in annotations_data['categories']
}

# Mapping of image file names to image metadata
image_name_to_metadata = {
    image['file_name']: image for image in annotations_data['images']
}

# Iterate over all images in the dataset
for image_name in os.listdir(images_path):
    # Load the image
    image_path = os.path.join(images_path, image_name)
    input_image = Image.open(image_path)

    # Get the image metadata
    if image_name not in image_name_to_metadata:
        print(f"Image {image_name} not found in annotations.")
        continue
    
    image_metadata = image_name_to_metadata[image_name]
    image_id = image_metadata['id']

    # Get the annotations for the current image
    annotations = image_to_annotations.get(image_id, [])

    # Extract objects and their bounding boxes
    objects = []
    boxes = []

    for annotation in annotations:
        category_id = annotation['category_id']
        category_name = category_id_to_name[category_id]

        # Add the object to the list
        objects.append(f"a {category_name}")

        # Normalize the bounding box coordinates
        bbox = annotation['bbox']  # [x_min, y_min, width, height]
        x_min, y_min, width, height = bbox
        image_width, image_height = image_metadata['width'], image_metadata['height']

        x_min_norm = x_min / image_width
        y_min_norm = y_min / image_height
        x_max_norm = (x_min + width) / image_width
        y_max_norm = (y_min + height) / image_height

        boxes.append([x_min_norm, y_min_norm, x_max_norm, y_max_norm])

    # Generate the prompt and phrases
    if len(objects) > 1:
        prompt = ", ".join(objects[:-1]) + f" and {objects[-1]}"
    else:
        prompt = objects[0]

    phrases = objects
    
    images = pipe(
    prompt=prompt,
    gligen_phrases=phrases,
    gligen_inpaint_image=input_image,
    gligen_boxes=boxes,
    gligen_scheduled_sampling_beta=1,
    output_type="pil",
    num_inference_steps=50,
    ).images
    images[0].save(f"/kaggle/working/output_images/{image_name}")

/opt/conda/lib/python3.10/site-packages/diffusers/pipelines/stable_diffusion_gligen/pipeline_stable_diffusion_gligen.py:747: FutureWarning: Accessing config attribute `sample_size` directly via 'AutoencoderKL' object attribute is deprecated. Please access 'sample_size' over 'AutoencoderKL's config object instead, e.g. 'unet.config.sample_size'.
  if gligen_inpaint_image.size != (self.vae.sample_size, self.vae.sample_size):
/opt/conda/lib/python3.10/site-packages/diffusers/pipelines/stable_diffusion_gligen/pipeline_stable_diffusion_gligen.py:748: FutureWarning: Accessing config attribute `sample_size` directly via 'AutoencoderKL' object attribute is deprecated. Please access 'sample_size' over 'AutoencoderKL's config object instead, e.g. 'unet.config.sample_size'.
  gligen_inpaint_image = self.target_size_center_crop(gligen_inpaint_image, self.vae.sample_size)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (92 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', a bicycle , a bicycle , a bicycle , a bicycle and a person']


  0%|          | 0/50 [00:00<?, ?it/s]